# Import libraries

The FitnessGram™ Pacer Test is a multistage aerobic capacity test that progressively gets more difficult as it continues. The 20 meter pacer test will begin in 30 seconds. Line up at the start. The running speed starts slowly, but gets faster each minute after you hear this signal. <i>beep</i> A single lap should be completed each time you hear this sound. <i>ding</i> Remember to run in a straight line, and run as long as possible. The second time you fail to complete a lap before the sound, your test is over. The test will begin on the word start. <b>On your mark, get ready, start.</b>

In [6]:
import ase.io # for reading *.xyz
import pandas as pd # for dataframes?
import numpy as np # for stats and such
import matplotlib.pyplot as plt # for plotting error distribution

# Reading and Importing Dataset

Next, blah blah blah

In [7]:
ase.io.read(filename="data_structures_errors.xyz", format="xyz")

KeyboardInterrupt: 